# Проект по Анализу данных Тихонова Илья и Тахунова Арсена

## About Dataset
📱 Smartphone Usage, Productivity & Lifestyle Analytics Dataset (50K Records)\
📌 Overview

This dataset contains 50,000 user records analyzing smartphone usage behavior and its relationship with productivity, sleep patterns, and stress levels. It is designed for machine learning, statistical analysis, and behavioral research.

Размерность: 50000 строк × 13 столбцов

📊 Dataset Features

The dataset includes the following variables:

User_ID – Unique identifier

Age – User age (18–60) (метрический)

Gender – Male, Female, Other (категориальный, номинальный)

Occupation – Student, Professional, Freelancer, Business Owner (категориальный, номинальный)

Device_Type – Android / iOS (категориальный, бинарный)

Daily_Phone_Hours – Average daily phone usage (метрический)

Social_Media_Hours – Daily time spent on social media (метрический)

Work_Productivity_Score – Productivity score (1–10) (категориальный, порядковый)

Sleep_Hours – Average sleep duration (метрический)

Stress_Level – Stress rating (1–10) (категориальный, порядковый)

App_Usage_Count – Number of apps used daily (метрический)

Caffeine_Intake_Cups – Daily caffeine consumption (метрический)

Weekend_Screen_Time_Hours – Screen time during weekends (метрический)



In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('Smartphone_Usage_Productivity_Dataset_50000.csv')
df

,User_ID,Age,Gender,Occupation,Device_Type,Daily_Phone_Hours,Social_Media_Hours,Work_Productivity_Score,Sleep_Hours,Stress_Level,App_Usage_Count,Caffeine_Intake_Cups,Weekend_Screen_Time_Hours
0,U1,58,Male,Professional,Android,1.3,6.7,6,8.8,4,42,1,8.7
1,U2,25,Male,Professional,Android,1.2,1.5,5,6.4,1,51,3,5.1
2,U3,19,Male,Student,iOS,5.3,5.7,5,9.0,4,14,5,6.3
3,U4,35,Female,Business Owner,iOS,5.8,2.5,2,5.7,3,36,6,12.8
4,U5,33,Male,Freelancer,Android,7.9,1.3,4,5.7,3,37,5,9.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,U49996,44,Male,Business Owner,Android,5.9,5.4,5,6.6,1,11,5,3.0
49996,U49997,42,Other,Business Owner,Android,2.9,7.4,9,6.3,2,20,4,6.2
49997,U49998,27,Female,Freelancer,iOS,1.4,2.5,4,6.7,9,39,4,5.1
49998,U49999,41,Female,Business Owner,iOS,8.9,3.0,6,5.5,2,51,0,9.2


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   User_ID                    50000 non-null  object 
 1   Age                        50000 non-null  int64  
 2   Gender                     50000 non-null  object 
 3   Occupation                 50000 non-null  object 
 4   Device_Type                50000 non-null  object 
 5   Daily_Phone_Hours          50000 non-null  float64
 6   Social_Media_Hours         50000 non-null  float64
 7   Work_Productivity_Score    50000 non-null  int64  
 8   Sleep_Hours                50000 non-null  float64
 9   Stress_Level               50000 non-null  int64  
 10  App_Usage_Count            50000 non-null  int64  
 11  Caffeine_Intake_Cups       50000 non-null  int64  
 12  Weekend_Screen_Time_Hours  50000 non-null  float64
dtypes: float64(4), int64(5), object(4)
memory usag

In [6]:
df.isnull().sum()

User_ID                      0
Age                          0
Gender                       0
Occupation                   0
Device_Type                  0
Daily_Phone_Hours            0
Social_Media_Hours           0
Work_Productivity_Score      0
Sleep_Hours                  0
Stress_Level                 0
App_Usage_Count              0
Caffeine_Intake_Cups         0
Weekend_Screen_Time_Hours    0
dtype: int64

### 3 пункт не делаем, так как все данные имеют корректные типы.

### 4. Создали новый признак "Avg_Weekly_Phone_Hours", который рассчитывает среднее количество часов, проведенных за телефоном в неделю, учитывая как ежедневное использование, так и использование в выходные дни. Формула для расчета: (Daily_Phone_Hours * 5 + Weekend_Screen_Time_Hours * 2) / 7. Результат округляем до одного десятичного знака

In [9]:
df['Avg_Weekly_Phone_Hours'] = ((df['Daily_Phone_Hours'] * 5 + df['Weekend_Screen_Time_Hours'] * 2) / 7).round(1)
df

,User_ID,Age,Gender,Occupation,Device_Type,Daily_Phone_Hours,Social_Media_Hours,Work_Productivity_Score,Sleep_Hours,Stress_Level,App_Usage_Count,Caffeine_Intake_Cups,Weekend_Screen_Time_Hours,Avg_Phone_Hours,Avg_Weekly_Phone_Hours
0,U1,58,Male,Professional,Android,1.3,6.7,6,8.8,4,42,1,8.7,3.4,3.4
1,U2,25,Male,Professional,Android,1.2,1.5,5,6.4,1,51,3,5.1,2.3,2.3
2,U3,19,Male,Student,iOS,5.3,5.7,5,9.0,4,14,5,6.3,5.6,5.6
3,U4,35,Female,Business Owner,iOS,5.8,2.5,2,5.7,3,36,6,12.8,7.8,7.8
4,U5,33,Male,Freelancer,Android,7.9,1.3,4,5.7,3,37,5,9.9,8.5,8.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,U49996,44,Male,Business Owner,Android,5.9,5.4,5,6.6,1,11,5,3.0,5.1,5.1
49996,U49997,42,Other,Business Owner,Android,2.9,7.4,9,6.3,2,20,4,6.2,3.8,3.8
49997,U49998,27,Female,Freelancer,iOS,1.4,2.5,4,6.7,9,39,4,5.1,2.5,2.5
49998,U49999,41,Female,Business Owner,iOS,8.9,3.0,6,5.5,2,51,0,9.2,9.0,9.0


### 5. Перекодируйте любой метрический признак в категориальный. Обоснуйте выбранные интервалы/группы.

##### Заменили количество часов сна на категории (короткий сон(до 6 часов), нормальный сон(от 6 до 8), длинный сон(от 8 часов)). Интервалы содержательно интерпретируемы. Удобно сравнивать группы по продуктивности и стрессу

In [11]:
bins = [-float('inf'), 6, 8, float('inf')]
labels = ['Short sleep', 'Normal sleep', 'Long sleep']

df['Sleep_Category'] = pd.cut(
    df['Sleep_Hours'],
    bins=bins,
    labels=labels,
    right=True,
    include_lowest=True)
df.head()

,User_ID,Age,Gender,Occupation,Device_Type,Daily_Phone_Hours,Social_Media_Hours,Work_Productivity_Score,Sleep_Hours,Stress_Level,App_Usage_Count,Caffeine_Intake_Cups,Weekend_Screen_Time_Hours,Avg_Phone_Hours,Avg_Weekly_Phone_Hours,Sleep_Category
0,U1,58,Male,Professional,Android,1.3,6.7,6,8.8,4,42,1,8.7,3.4,3.4,Long sleep
1,U2,25,Male,Professional,Android,1.2,1.5,5,6.4,1,51,3,5.1,2.3,2.3,Normal sleep
2,U3,19,Male,Student,iOS,5.3,5.7,5,9.0,4,14,5,6.3,5.6,5.6,Long sleep
3,U4,35,Female,Business Owner,iOS,5.8,2.5,2,5.7,3,36,6,12.8,7.8,7.8,Short sleep
4,U5,33,Male,Freelancer,Android,7.9,1.3,4,5.7,3,37,5,9.9,8.5,8.5,Short sleep


### 6. Чётко и детально сформулируйте цель и задачи исследования.

### Цель исследования

**Выявить, как показатели цифрового поведения (экранное время, соцсети, количество приложений, кофеин), сон и тип устройства связаны с уровнем продуктивности и стресса пользователей, а также выделить типичные поведенческие профили пользователей с помощью кластерного анализа.**

### Задачи исследования
- Описательная статистика: средние/медианы/квантили, распределения и вариативность по всем числовым признакам; частоты по категориальным.
- Сравнение групп пользователей по показателям использования/сна/кофеина/стресса/продуктивности по Gender, Occupation, Device_Type и возрастным группам.
- Рассчитать описательные статистики по всей выборке и по группам.
- Исследовать корреляции между числовыми признаками.
3. Построить визуализации для выявления зависимостей.
4. Построить сводные таблицы и проверить связи между категориальными признаками (в т.ч. χ²).
5. Построить модель линейной регрессии для `Work_Productivity_Score`.
6. Построить модель бинарной регрессии для классификации уровня стресса.
7. Выполнить кластерный анализ и интерпретировать кластеры.
8. Проверить исследовательские гипотезы и сформулировать итоговые выводы.

### 7. Сформулируйте не менее 10 исследовательских гипотез, которые будут проверены в ходе анализа данных. По каждой гипотезе уточните и обоснуйте выбор признаков и методов анализа данных, которые будут использованы для её проверки.

#### **H1.** Чем больше `Daily_Phone_Hours`, тем ниже `Work_Productivity_Score`.

* **Признаки:** `Daily_Phone_Hours`, `Work_Productivity_Score`
* **Почему:** проверка базовой гипотезы о перегрузке экранным временем
* **Методы:** scatterplot, Spearman/Pearson, линейная регрессия

---

#### **H2.** Чем больше `Social_Media_Hours`, тем выше `Stress_Level`.

* **Признаки:** `Social_Media_Hours`, `Stress_Level`
* **Почему:** соцсети могут усиливать стресс/информационную перегрузку
* **Методы:** корреляция Spearman, boxplot по группам, логистическая регрессия (для High_Stress)

---

#### **H3.** Пользователи с `Sleep_Category = Short sleep` имеют более низкую продуктивность, чем группы `Normal sleep` и `Long sleep`.

* **Признаки:** `Sleep_Category`, `Work_Productivity_Score`
* **Почему:** сон — ключевой фактор когнитивной эффективности
* **Методы:** boxplot, ANOVA (или Kruskal–Wallis), post-hoc сравнения

---

#### **H4.** Пользователи с недостатком сна имеют более высокий стресс.

* **Признаки:** `Sleep_Category`, `Stress_Level` или `High_Stress`
* **Почему:** содержательно связано с темой сна
* **Методы:** сводная таблица, χ² (для категорий), логистическая регрессия

---

#### **H5.** `Caffeine_Intake_Cups` положительно связан с `Stress_Level`.

* **Признаки:** `Caffeine_Intake_Cups`, `Stress_Level`
* **Почему:** кофеин может быть маркером нагрузки/усталости
* **Методы:** Spearman, boxplot, логистическая регрессия

---

#### **H6.** `Weekend_Screen_Time_Hours` выше, чем `Daily_Phone_Hours` в будни (в среднем по выборке).

* **Признаки:** `Weekend_Screen_Time_Hours`, `Daily_Phone_Hours`
* **Почему:** проверка поведенческой разницы будни/выходные
* **Методы:** описательная статистика, paired-style сравнение (если трактовать как две меры на пользователя), histogram/boxplot

---

#### **H7.** Доля соцсетей в общем экранном времени отрицательно связана с продуктивностью.

* **Новый признак:** `SocialMedia_Share = Social_Media_Hours / Daily_Phone_Hours`
* **Признаки:** `SocialMedia_Share`, `Work_Productivity_Score`
* **Почему:** лучше отражает структуру использования, чем абсолютное время
* **Методы:** инженерия признака, корреляция, линейная регрессия

---

#### **H8.** `App_Usage_Count` положительно связан с `Daily_Phone_Hours`.

* **Признаки:** `App_Usage_Count`, `Daily_Phone_Hours`
* **Почему:** больше приложений → больше времени взаимодействия
* **Методы:** scatterplot, Pearson/Spearman, корреляция

---

#### **H9.** Между `Device_Type` (Android/iOS) есть различия по `Work_Productivity_Score`.

* **Признаки:** `Device_Type`, `Work_Productivity_Score`
* **Почему:** в задании есть отдельный блок сравнения по устройствам
* **Методы:** boxplot/violin, t-test или Mann–Whitney (после проверки распределений)

---

#### **H10.** Между `Occupation` и уровнем стресса существуют различия.

* **Признаки:** `Occupation`, `Stress_Level` / `High_Stress`
* **Почему:** разные сценарии использования смартфона по профессиям
* **Методы:** boxplot, ANOVA/Kruskal; сводная таблица + χ² (если стресс категоризовать)

---

#### **H11.** После учета сна и соцсетей влияние общего экранного времени на продуктивность уменьшается.

* **Признаки:** `Daily_Phone_Hours`, `Sleep_Hours`, `Social_Media_Hours`, `Work_Productivity_Score`
* **Почему:** проверка независимого вклада факторов
* **Методы:** множественная линейная регрессия (сравнение коэффициентов / частичных эффектов)

---

#### **H12.** Выделенные кластеры пользователей значимо различаются по продуктивности и стрессу.

* **Признаки:** кластерные признаки + `Work_Productivity_Score`, `Stress_Level`
* **Почему:** это цель кластерного анализа
* **Методы:** KMeans, silhouette/elbow, затем ANOVA/Kruskal и описательная интерпретация

#### 8. Постройте не менее 5 графиков по имеющимся данным. У графиков должны быть подписи осей, названия и легенда. Каждый график должен отражать некоторые тенденции или взаимосвязи, наблюдаемые в данных. Под каждым графиком нужно написать комментарий, описывающий тенденции или взаимосвязи, которые удалось выявить в результате его построения.

#### 9. Постройте не менее 3-х сводных таблиц. Под каждой таблицей нужно написать комментарий, описывающий тенденции или взаимосвязи, которые удалось выявить в результате её построения. Проведите тест Хи-квадрат, чтобы оценить наличие взаимосвязи между рассмотренными признаками (проверьте подходят ли данные для проведения теста, сформулируйте гипотезы для теста, сделайте выводы по результатам проведения теста).

#### 10. Рассчитайте описательные статистики, характеризующие центральную тенденцию и разброс значений относительно меры центральной тенденции по всей выборке и по отдельным группам наблюдений. Под каждым выводом напишите комментарий, описывающий тенденции или взаимосвязи, которые удалось выявить в результате расчётов.

#### 11. Посчитайте корреляции между числовыми данными и интерпретируйте полученные результаты. Сделайте выводы о специфике взаимосвязей между рассмотренными признаками.

#### 12. Постройте модель линейной регрессии. В модели должно быть не менее 5 предикторов. Детально опишите полученную модель. Сформулируйте содержательные выводы по результатам проведенных расчётов. Нужно уточнить особенности взаимосвязей, наблюдаемых между каждым предиктором и зависимой переменной.

#### 13. Постройте модель бинарной регрессии. В модели должно быть не менее 5 предикторов. Детально опишите полученную модель. Сформулируйте содержательные выводы по результатам проведенных расчётов. Нужно уточнить особенности взаимосвязей, наблюдаемых между каждым предиктором и зависимой переменной.

#### 14. Проведите кластерный анализ данных. Сформулируйте содержательные выводы по результатам проведенных расчётов. Подробно опишите каждый из кластеров, придумайте для них названия.

#### 15. Сделайте выводы относительно того, какие исследовательские гипотезы нашли подтверждение в ходе исследования, а какие были отвергнуты и почему.

#### 16. Приведите доводы в пользу того, что цель и задачи исследования были реализованы. Сформулируйте общие выводы по результатам анализа данных. Какие выводы показались вам ожидаемыми, а какие неожиданными и почему?